In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import dok_matrix
from tqdm import tqdm

In [2]:
data_dir = '/opt/ml/movie-recommendation/data/train/'
data = pd.read_csv(data_dir + 'train_ratings.csv')
data = data[['user', 'item']]

In [3]:
user_group_dfs = data.groupby(by='user')['item']

In [4]:
train_dfs = []
test_dfs  = []

items = set(data.loc[:, 'item'])

for u, u_items in tqdm(user_group_dfs):
    num_data = len(u_items)
    num_test = int(num_data*0.2)

    train_idx = np.random.choice(num_data, num_data-num_test, replace=False)
    test_idx = [idx for idx in range(num_data) if idx not in train_idx]

    train_df = pd.DataFrame({'user':[u]*len(train_idx), 'item':u_items.values[train_idx]})
    test_df = pd.DataFrame({'user':[u]*len(test_idx), 'item':u_items.values[test_idx], 'rating': [1]*len(test_idx)})

    num_negs = len(test_idx)*2 if len(test_idx) > 10 else 10
    neg_items = np.random.choice(list(items - set(u_items)), num_negs, replace=False)
    neg_df = pd.DataFrame({'user': [u]*num_negs, 'item': neg_items, 'rating': [0]*num_negs})

    train_dfs.append(train_df)
    test_dfs.extend([test_df, neg_df])

train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)

100%|██████████| 31360/31360 [01:41<00:00, 309.99it/s]


In [5]:
data_dir = '/opt/ml/movie-recommendation/data/train/bpr/'
train_df.to_csv(data_dir+'train.csv')
test_df.to_csv(data_dir+'valid.csv')